# Basics

This example will walk you through getting started with py-allspice and the basics of using it.

## Installation

First, you'll need to install py-allspice. You can do that by running this cell:

In [ ]:
# This is a magic cell, and only runs in Jupyter notebooks! Don't copy this into a .py file.
%pip install py-allspice

Or alternatively, you can install it from the command line using:

```bash
pip install py-allspice
```

## Creating a client

All operations in py-allspice require an AllSpice client object. Before you can make a client, you
need an AllSpice Hub Auth Token. [See this guide for more information.](https://allspice.document360.io/docs/how-to-create-an-allspice-authentication-application-access-token)

Once you have a token, you can make a client like this:

In [ ]:
from allspice import AllSpice

# Replace the text in the quotes with your token.
client = AllSpice(token_text="<YOUR TOKEN HERE>")

There are a few other options you can pass to the client constructor. For example, you can specify
the URL of the AllSpice Hub you want to connect to,

In [ ]:
client = AllSpice(
    allspice_hub_url="https://my.selfhosted.example",
    auth=("username", "password"),
    log_level="DEBUG",
)

Note that if you specify a username and password, you shouldn't specify a token. In general, prefer
using tokens over usernames and passwords, as they are more secure and can be revoked at any time.
If you are using a token in a python script that is checked into a repository, you should use an
environment variable to store the token, and then read it from the environment in your script:

In [ ]:
import os

client = AllSpice(token_text=os.environ["ALLSPICE_TOKEN"])

In this example, we're using the `ALLSPICE_TOKEN` environment variable. You can use any environment
variable you want, as long as you set it to the token text.

## Getting Objects from AllSpice

AllSpice has a number of different concepts, such as "Organizations", "Repositories", "Issues",
"Design Reviews" and so on. Each of these concepts has a corresponding object in py-allspice, and
you can perform operations on these objects to interact with AllSpice. For example, you can get all
the organizations you have access to like this:

In [ ]:
organizations = client.get_orgs()

for org in organizations:
    print(org.username)

Typically, you want to get a single object you know of, like a repository. Here is how you can get
a repository by its name:

In [ ]:
from allspice import Repository

repo = Repository.request(client, "my-org", "my-repo")

print(repo.name)

Almost every object has a similar `request` method that you can use to get it, such as getting an
issue by its number:

In [ ]:
from allspice import Issue

issue = Issue.request(client, "my-org", "my-repo", "1")

print(f"#{issue.number}: {issue.title}")

Here is a list of all the objects in py-allspice:

- User
- Organization
- Team
- Repository
- Branch
- NotFoundException
- AlreadyExistsException
- Issue
- Milestone
- Commit
- Comment
- Content
- DesignReview

To explore how you can get these objects, you can use the `help` function in python:

In [ ]:
from allspice import User

help(User.request)

## Working with Objects

Once you have an object, you can access its properties to get information about it. For example, you
can get the name of a repository:

In [ ]:
print(repo.name)

You can get a list of all properties and methods on an object by using the `dir` function in python:

In [ ]:
dir(repo)

You can also perform operations on objects. For example, you can get all the issues in a repository:

In [ ]:
issues = repo.get_issues()

for issue in issues:
    print(f"#{issue.number}: {issue.title}")

Or create an issue:

In [ ]:
new_issue = repo.create_issue(title="My New Issue", description="This is the body of my new issue.")

## Editing and Deleting Objects

Most types of objects can be edited and deleted. For editing, you can simply set the properties you
want to change, and then call the `commit` method on the object:

In [ ]:
new_issue.title = "My New Issue (Edited)"
new_issue.commit()

new_issue_again = Issue.request(client, "my-org", "my-repo", new_issue.number)
print(new_issue_again.title)

Not all properties can be edited. If there is a guide for editing a specific type of object, it will
mention which properties can be edited.

To delete an object, you can call the `delete` method on it:

In [ ]:
from allspice import NotFoundException

new_issue.delete()

try:
    Issue.request(client, "my-org", "my-repo", new_issue.number)
except NotFoundException:
    print("Issue was deleted!")

## Errors

If an error occurs while performing an operation, py-allspice will raise an exception. For example,
if you try to get an issue that doesn't exist, you will get a `NotFoundException`, as seen above.
In general, the Error will be descriptive about what went wrong, and how you can fix it. One common
error that might be confusing is a `NotFoundException` when trying to get an object that you know
exists. This can happen if the API token you're using don't have access to the object. To fix this,
double check that the user or token has access to the object.
